# Build Reference Standard Masks

In this notebook there is all the code needed to compute the metrics used in this workf for evaluation. This is the notebook used to generate the metric results CSV which is later used for comparisons and evaluation.

Requirements not available in this repository:
* To be able to run this code one needs to run it in an eviroment with the python packages: __numpy__, __pandas__.
* To be able to run this code one needs the __MRI sequences__ for each subject wich is not uploaded.

In [1]:
import os
import pandas as pd

import metrics
from utils import getImages, get_subject_path
from constants import *

In [2]:
# List all the available subjects
subject_codes = os.listdir('../results/')

In [3]:
data = []

# Loop over al the teams
for team_name in TEAM_NAMES:
    
    # Loop over all subject results
    for subject_code in subject_codes:
        
        # Catch if any results are wrong or missing
        try:
            
            # Read result and test image
            subject_path = get_subject_path(subject_code)
            test_image, result_image = getImages(
                f'{subject_path}input/wmh_gauss.nii.gz',
                f'{subject_path}{team_name}/result.nii.gz'
            )

            # Compute metrics
            dsc = metrics.getDSC(test_image, result_image)
            h95 = metrics.getHausdorff(test_image, result_image)
            vol = metrics.getVOL(test_image, result_image)    
            recall, f1, nWMH = metrics.getLesionDetection(test_image, result_image)

            # Data to insert
            data.append({
                'Subject Code': subject_code,
                'Team Name': team_name,
                DSC: dsc,
                HDF: h95,
                VOL: vol,
                REC: recall,
                F1: f1,
                DWMH: int(nWMH * recall),
                NWMH: int(nWMH)
            })
        except RuntimeError as e:
            print(e)
            data.append({
                'Subject Code': subject_code,
                'Team Name': team_name,
                DSC: None,
                HDF: None,
                VOL: None,
                REC: None,
                F1: None,
                DWMH: None,
                NWMH: None
            })

    # Create pandas DataFrame and save as CSV
    results_df = pd.DataFrame(data)
    results_df.to_csv(f'../metric_results.csv', sep=';')